# ZeissManager Example Notebook

In [1]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time
from pathlib import Path

In [7]:
# --- Set input data path ---
main_path = "../../pymif_test_data/zeiss"
dataset_name = "2TR-6M-3T-3C-5Z-64YX_stitched.czi"
czi_path = f"{main_path}/{dataset_name}"  # Update this to your real path


In [8]:
# --- Initialize manager ---
dataset = mm.ZeissManager(path=czi_path, scene_index=1)

In [9]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

SIZE: [(3, 3, 5, 123, 125)]
SCALES: [(3.99, 1.3258281249999997, 1.3258281249999997)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 30.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['AF594-T1', 'AF488-T2', 'T-PMT-T2']
CHANNEL_COLORS: ['#FF0100', '#00FF33', '#D3D3D3']
DTYPE: uint16
PLANE_FILES: 2TR-6M-3T-3C-5Z-64YX_stitched
AXES: tczyx
CHUNK SIZE: (1, 1, 5, 123, 125)


In [10]:
dataset.data[0]

dask.array<getitem, shape=(3, 3, 5, 123, 125), dtype=uint16, chunksize=(1, 1, 5, 123, 125), chunktype=numpy.ndarray>

In [11]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
viewer = dataset.visualize()

/home/gritti/.local/lib/python3.10/site-packages/dask/array/core.py:5085: RuntimeWarning: overflow encountered in scalar multiply
  result = function(*args, **kwargs)


In [12]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels, generate lower resolution and load it into memory
viewer = dataset.visualize( start_level = 2, in_memory = True )
# NOTE: This fails because the dataset only has 1 resolution layer

ValueError: start_level=2 is out of bounds for available 1 levels.

In [ ]:
# --- Build pyramid if not already ---
dataset.build_pyramid(num_levels=3, downscale_factor=2)

# --- Optional: Write to OME-Zarr format ---
output_zarr_path = f"{main_path}/{Path(dataset_name).stem}_{dataset.scene_name}.zarr"
dataset.write(output_zarr_path)

In [ ]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

# Now we use the ZarrManager to load the new data and display it

In [ ]:
input_zarr_path = output_zarr_path

In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
# Here, we use the same chunks.
zarr_dataset = mm.ZarrManager(path=input_zarr_path)

In [ ]:
# --- Show metadata summary ---
for i in zarr_dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")
print("CHUNK SIZE:", zarr_dataset.chunks)

In [ ]:
zarr_dataset.data

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster but not interactive.
start = time.time()
viewer = zarr_dataset.visualize()
print(f"{time.time()-start}")

In [ ]:
start = time.time()
viewer = zarr_dataset.visualize(                        
                        start_level = 0, 
                        stop_level = 2, 
                        in_memory = True 
)
print(f"{time.time()-start}")